In [4]:
import pandas as pd
import os
from math import cos, asin, sqrt, pi
import datetime

# Part 1: Data Processing

In [ ]:
TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
# get HTML from main page
content = requests.get(TAXI_URL)
soup = bs4.BeautifulSoup(content.text, 'html.parser')

In [ ]:
# find all links on the main page
pattern = r"yellow_tripdata_20(0[0-9]|1[0-5])\-\d{2}.csv$"
links = [a['href'] for a in soup.find_all('a')]x

In [ ]:
# use regex to get only the yellow taxi links from 2009 - june 2015
new_list = []

for link in links:
    if re.search(pattern, link) is not None:
        mo_1 = link[-6]
        mo_2 = link[-5]
        if mo_1 == "1":
            new_list.append(link)
        elif int(mo_2) <= 6:
            new_list.append(link)

In [ ]:
# write csvs to files
i = 0

for link in new_list:
    print(i)
    i += 1
    req = requests.get(link)
    url_content = req.content
    
    csv_file = open('downloaded.csv', 'ab')
    csv_file.write(url_content)
    csv_file.close()

In [ ]:
# convert csv to df
import pandas as pd
entire_df = pd.read_csv("downloaded.csv")

**ADD CODE TO SAMPLE DATA**

In [21]:
# Load the sampled data
df = pd.read_csv('sampled.csv')

/Users/scottqin/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1,2,3,12,20,21,27,30,35,36,42,45,53,54) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [22]:
# We will clean the sampled data year by year. There are columns with different names showing the same types of data.
# We need to merge these columns together and delete the irrelevant columns
# From the given sample table we know there are 8 useful columns.

In [23]:
pd.set_option('display.max_columns', None)

In [24]:
# create a new datafram called cleaned
cleaned = pd.DataFrame()
# create new columns
pickup_datetime = []
fare_amount = []
pickup_longitude = []
pickup_latitude = []
dropoff_longitude = []
dropoff_latitude = []
passenger_count = []

In [25]:
# clean the data from 2009
temp = df.iloc[0:14400]
pickup_datetime += list(temp['Trip_Pickup_DateTime'])
fare_amount += list(temp['Fare_Amt'])
pickup_longitude += list(temp['Start_Lon'])
pickup_latitude += list(temp['Start_Lat'])
dropoff_longitude += list(temp['End_Lon'])
dropoff_latitude += list(temp['End_Lat'])
passenger_count += list(temp['Passenger_Count'])

In [26]:
# clean the data from 2010
temp = df.iloc[14400:28800]
pickup_datetime += list(temp['pickup_datetime'])
fare_amount += list(temp['fare_amount'])
pickup_longitude += list(temp['pickup_longitude'])
pickup_latitude += list(temp['pickup_latitude'])
dropoff_longitude += list(temp['dropoff_longitude'])
dropoff_latitude += list(temp['dropoff_latitude'])
passenger_count += list(temp['passenger_count'])

In [27]:
# clean the data from 2011
temp = df.iloc[28800:43200]
pickup_datetime += list(temp['pickup_datetime'])
fare_amount += list(temp['fare_amount'])
pickup_longitude += list(temp['pickup_longitude'])
pickup_latitude += list(temp['pickup_latitude'])
dropoff_longitude += list(temp['dropoff_longitude'])
dropoff_latitude += list(temp['dropoff_latitude'])
passenger_count += list(temp['passenger_count'])

In [28]:
# clean the data from 2012
temp = df.iloc[43200:57600]
pickup_datetime += list(temp['pickup_datetime'])
fare_amount += list(temp['fare_amount'])
pickup_longitude += list(temp['pickup_longitude'])
pickup_latitude += list(temp['pickup_latitude'])
dropoff_longitude += list(temp['dropoff_longitude'])
dropoff_latitude += list(temp['dropoff_latitude'])
passenger_count += list(temp['passenger_count'])

In [29]:
# clean the data from 2013
temp = df.iloc[57600:72000]
pickup_datetime += list(temp['pickup_datetime'])
fare_amount += list(temp['fare_amount'])
pickup_longitude += list(temp['pickup_longitude'])
pickup_latitude += list(temp['pickup_latitude'])
dropoff_longitude += list(temp['dropoff_longitude'])
dropoff_latitude += list(temp['dropoff_latitude'])
passenger_count += list(temp['passenger_count'])

In [30]:
# clean the data from 2014
temp = df.iloc[72000:86400]
pickup_datetime += list(temp['pickup_datetime'])
fare_amount += list(temp['fare_amount'])
pickup_longitude += list(temp['pickup_longitude'])
pickup_latitude += list(temp['pickup_latitude'])
dropoff_longitude += list(temp['dropoff_longitude'])
dropoff_latitude += list(temp['dropoff_latitude'])
passenger_count += list(temp['passenger_count'])

In [31]:
# clean the data from 2015
temp = df.iloc[86400:]
pickup_datetime += list(temp['tpep_pickup_datetime'])
fare_amount += list(temp['fare_amount'])
pickup_longitude += list(temp['pickup_longitude'])
pickup_latitude += list(temp['pickup_latitude'])
dropoff_longitude += list(temp['dropoff_longitude'])
dropoff_latitude += list(temp['dropoff_latitude'])
passenger_count += list(temp['passenger_count'])


In [32]:
# add all the clean columns to the clean table
cleaned['pickup_datetime'] = pickup_datetime
cleaned['fare_amount'] = fare_amount
cleaned['pickup_longitude'] = pickup_longitude
cleaned['pickup_latitude'] = pickup_latitude
cleaned['dropoff_longitude']= dropoff_longitude
cleaned['dropoff_latitude'] = dropoff_latitude
cleaned['passenger_count'] = passenger_count

In [33]:
# exclude data outside of nyc
cleaned = cleaned.loc[(cleaned['pickup_longitude'] > -74.24233) & (cleaned['pickup_longitude'] < -73.717047)]
cleaned = cleaned.loc[(cleaned['dropoff_longitude'] > -74.24233) & (cleaned['dropoff_longitude'] < -73.717047)]
cleaned = cleaned.loc[(cleaned['pickup_latitude'] > 40.560445) & (cleaned['pickup_latitude'] < 40.908524)]
cleaned = cleaned.loc[(cleaned['dropoff_latitude'] > 40.560445) & (cleaned['dropoff_latitude'] < 40.908524)]

In [34]:
# define a function to calculate the distance between coordinates
def distance(lat1, lon1, lat2, lon2):
    p = pi/180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    return 12742 * asin(sqrt(a))

In [35]:
# testing the function
distance(40.751500, -73.984090, 40.770418, -73.864958)

10.251851855874952

In [36]:
# apply the function to table and add a new distance column
cleaned['distance'] = cleaned[['pickup_latitude','pickup_longitude','dropoff_latitude', 
                              'dropoff_longitude']].apply(lambda x: distance(*x), axis=1)

In [37]:
# define a function to extract the hour number
def hour_extract(x):
    return x[11:13]

In [38]:
# apply the function to table and add a new hour column
cleaned['hour'] = cleaned[['pickup_datetime']].apply(lambda x: int(hour_extract(*x)), axis=1)

In [39]:
uber = pd.read_csv('uber_rides_sample.csv')

In [40]:
# add distance column to uber table
uber['distance'] = uber[['pickup_latitude','pickup_longitude','dropoff_latitude', 
                              'dropoff_longitude']].apply(lambda x: distance(*x), axis=1)

In [41]:
# define a day conversion function
def day_conv(x):
    return datetime.datetime.strptime(x[:10], '%Y-%m-%d').strftime('%A')

In [42]:
# testing the function
day_conv(uber['pickup_datetime'][0])

'Thursday'

In [43]:
# apply the function to uber table and add a new day column
uber['day'] = uber[['pickup_datetime']].apply(lambda x: day_conv(*x), axis=1)

In [45]:
os.makedirs('Github/4501-Project', exist_ok=True)  
cleaned.to_csv('cleaned.csv') 
uber.to_csv('uber.csv') 

# Part 2: Storing Data

In [1]:
from sqlalchemy import create_engine
engine = create_engine(f'sqlite:///project.db', echo=True)
sqlite_connection = engine.connect()

In [4]:
# create yellow taxi sql table
yellow_taxi = pd.read_csv("cleaned.csv")
yellow_taxi.to_sql("yellow_taxi", engine)

2022-04-30 15:59:39,453 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("yellow_taxi")
2022-04-30 15:59:39,454 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-30 15:59:39,455 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("yellow_taxi")
2022-04-30 15:59:39,455 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-30 15:59:39,457 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-30 15:59:39,458 INFO sqlalchemy.engine.Engine 
CREATE TABLE yellow_taxi (
	"index" BIGINT, 
	"Unnamed: 0" BIGINT, 
	pickup_datetime TEXT, 
	fare_amount FLOAT, 
	pickup_longitude FLOAT, 
	pickup_latitude FLOAT, 
	dropoff_longitude FLOAT, 
	dropoff_latitude FLOAT, 
	passenger_count FLOAT, 
	distance FLOAT, 
	hour BIGINT
)


2022-04-30 15:59:39,458 INFO sqlalchemy.engine.Engine [no key 0.00040s] ()
2022-04-30 15:59:39,460 INFO sqlalchemy.engine.Engine CREATE INDEX ix_yellow_taxi_index ON yellow_taxi ("index")
2022-04-30 15:59:39,460 INFO sqlalchemy.engine.Engine [no key 0.00040s] ()
2022-04-30 15:59

In [ ]:
# create uber sql table

In [ ]:
# create weather sql table

In [ ]:
def write_query_to_file(query, outfile):
    raise NotImplemented()

In [5]:
QUERY_1 = """
SELECT "hour", count(hour)
    FROM "yellow_taxi"
    GROUP BY "hour"
"""

engine.execute(QUERY_1).fetchall()
# hour 19 (7pm) is the most popular time of day to take a yellow taxi

2022-04-30 15:59:45,208 INFO sqlalchemy.engine.Engine 
SELECT "hour", count(hour)
    FROM "yellow_taxi"
    GROUP BY "hour"

2022-04-30 15:59:45,212 INFO sqlalchemy.engine.Engine [raw sql] ()


[(0, 3057),
 (1, 2225),
 (2, 1689),
 (3, 1218),
 (4, 879),
 (5, 766),
 (6, 1558),
 (7, 2692),
 (8, 3584),
 (9, 3590),
 (10, 3477),
 (11, 3579),
 (12, 3730),
 (13, 3815),
 (14, 3943),
 (15, 3748),
 (16, 3344),
 (17, 3883),
 (18, 4748),
 (19, 4761),
 (20, 4567),
 (21, 4565),
 (22, 4203),
 (23, 3805)]

# Part 3: Understanding Data

# Part 4: Visualizing Data